
# Interactive Moon Phase and Lottery Win Analysis

This notebook presents an interactive visualization showing the relationship between the moon's phase (represented as "Moon Age Positive") and lottery win percentages. The aim is to explore any potential correlations between the lunar cycle and lottery outcomes.

## Methodology

- The dataset includes dates, moon ages, and win percentages for lottery draws.
- A continuous line represents the moon's age over time.
- Points overlaid on this line represent win percentages for each day, color-coded and sized based on their value.


In [1]:
print("Hello, world!")


Hello, world!
